# Sort background files automatically

Look for the bkg files in the `BKGS` folder.
Look at the timestamp of creation of each file and match them to the `MAP` folder it belongs.

*It doesn't work if you modify the `HYPCard.bin` files (modify the date of writing.*

In [57]:
import numpy as np
import os, glob, shutil
import datetime

In [65]:
root = r'G:\My Drive\PhD\projects\external_measurements\20211118_CL_Mn_doped_NCs_Sascha'

In [113]:
#Get the creation time of a file from Windows metadata (does not work well)
def get_creation_dates(path):
    abs_t = os.path.getmtime(path)
    t = datetime.datetime.fromtimestamp(abs_t)
    return abs_t, t

# Get the creation time of a file from file name (does not work well)
import re, dateutil
def get_creation_dates_from_str(path):
    m = re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2}\-[0-9]{2}h[0-9]{2}m[0-9]{2}s)", path)
    t = dateutil.parser.parse(m.group(0))
    abs_t = datetime.datetime.timestamp(t)
    return abs_t, t

In [114]:
# Get a list of the bkgs files and their creation time
bkgs_folder = os.path.join(root, 'BKGS')

bkgs = glob.glob(os.path.join(bkgs_folder, '*.txt'))
bkgs = [bkg for bkg in bkgs if '-V' not in bkg]

bkgs_dict = {}
for bkg in bkgs:
    abs_t, t = get_creation_dates_from_str(bkg)
    bkgs_dict[bkg] = {'t_stamp': abs_t, 't': t}

bkgs_dict

{'G:\\My Drive\\PhD\\projects\\external_measurements\\20211118_CL_Mn_doped_NCs_Sascha\\BKGS\\BG-bkg-50--50ms-2021-11-18-10h40m01s.txt': {'t_stamp': 1637232001.0,
  't': datetime.datetime(2021, 11, 18, 10, 40, 1)},
 'G:\\My Drive\\PhD\\projects\\external_measurements\\20211118_CL_Mn_doped_NCs_Sascha\\BKGS\\BG-bkg-100ms-100ms-2021-11-18-11h41m04s.txt': {'t_stamp': 1637235664.0,
  't': datetime.datetime(2021, 11, 18, 11, 41, 4)},
 'G:\\My Drive\\PhD\\projects\\external_measurements\\20211118_CL_Mn_doped_NCs_Sascha\\BKGS\\BG-bkg-250-250ms-2021-11-18-11h50m29s.txt': {'t_stamp': 1637236229.0,
  't': datetime.datetime(2021, 11, 18, 11, 50, 29)},
 'G:\\My Drive\\PhD\\projects\\external_measurements\\20211118_CL_Mn_doped_NCs_Sascha\\BKGS\\BG-bkg-50ms-50ms-2021-11-18-10h59m39s.txt': {'t_stamp': 1637233179.0,
  't': datetime.datetime(2021, 11, 18, 10, 59, 39)},
 'G:\\My Drive\\PhD\\projects\\external_measurements\\20211118_CL_Mn_doped_NCs_Sascha\\BKGS\\BG-bkg-500-500ms-2021-11-18-12h24m59s.txt': 

In [119]:
# Find all the HYP maps and their creation time too
hyp_maps = glob.glob(os.path.join(root,'*/HYPCard.bin'))
hyp_maps

hyp_maps_dict = {}
for hyp_map in hyp_maps:
    abs_t, t = get_creation_dates(hyp_map)
    hyp_maps_dict[hyp_map] = {'t_stamp': abs_t, 't': t}

hyp_maps_dict

{'G:\\My Drive\\PhD\\projects\\external_measurements\\20211118_CL_Mn_doped_NCs_Sascha\\HYP-MAP1\\HYPCard.bin': {'t_stamp': 1637235816.0,
  't': datetime.datetime(2021, 11, 18, 11, 43, 36)},
 'G:\\My Drive\\PhD\\projects\\external_measurements\\20211118_CL_Mn_doped_NCs_Sascha\\HYP-MAP3\\HYPCard.bin': {'t_stamp': 1637237322.0,
  't': datetime.datetime(2021, 11, 18, 12, 8, 42)},
 'G:\\My Drive\\PhD\\projects\\external_measurements\\20211118_CL_Mn_doped_NCs_Sascha\\HYP-MAP5\\HYPCard.bin': {'t_stamp': 1637240404.0,
  't': datetime.datetime(2021, 11, 18, 13, 0, 4)},
 'G:\\My Drive\\PhD\\projects\\external_measurements\\20211118_CL_Mn_doped_NCs_Sascha\\HYP-MAP8\\HYPCard.bin': {'t_stamp': 1637247172.0,
  't': datetime.datetime(2021, 11, 18, 14, 52, 52)},
 'G:\\My Drive\\PhD\\projects\\external_measurements\\20211118_CL_Mn_doped_NCs_Sascha\\HYP-MAP7\\HYPCard.bin': {'t_stamp': 1637246142.0,
  't': datetime.datetime(2021, 11, 18, 14, 35, 42)},
 'G:\\My Drive\\PhD\\projects\\external_measurements\

In [120]:
for hyp_map in hyp_maps:
    folder = os.path.dirname(hyp_map)
    to_remove = glob.glob(os.path.join(folder, 'BG*.txt'))
    if len(to_remove) == 1:
        os.remove(to_remove[0])
    else:
        [os.remove(f) for f in to_remove]

In [121]:
# Get the time list of the bkg files
bkgs_ts = [value['t_stamp'] for value in bkgs_dict.values()]

for hyp_map_path, hyp_map_ts in hyp_maps_dict.items():
    # Find closest file
    t_hypmap = hyp_map_ts['t_stamp']
    t_diff = [t_hypmap - bkgs_t for bkgs_t in bkgs_ts]
    t_min = min([i for i in t_diff if i > 0])
    index = t_diff.index(t_min)
    # print(index)
    # print(t_min)
    # print(datetime.datetime.fromtimestamp(bkgs_ts[index]),
    #       datetime.datetime.fromtimestamp(t_hypmap))


    # Copy and paste bkg file in original folder
    bkg_path = [k for k in bkgs_dict.keys()][index]
    new_path = os.path.join(os.path.dirname(hyp_map_path), os.path.basename(bkg_path))
    shutil.copy2(bkg_path, new_path)
